# Introduction

**Related notebooks**

Utility script notebook (with additional functions, aggregators):

https://www.kaggle.com/andreynesterov/home-credit-baseline-data

Inference notebook:

https://www.kaggle.com/andreynesterov/home-credit-baseline-inference

# Dependencies

In [1]:
!pip install --no-index -Uq --find-links=/kaggle/input/lightautoml-038-dependencies lightautoml==0.3.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.1.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2024.1.0 which is incompatible.
d

In [2]:
import os
import gc
from glob import glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import lightgbm as lgb
import torch
import torch.nn as nn

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [3]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import mean_squared_error

# Data collection

In [4]:
### from https://www.kaggle.com/code/batprem/home-credit-risk-mode-utility-scripts
# ざっくり言うとpandasテーブルのデータを軽くしてくれる関数。ブラックボックスに利用して良さそうな気がする

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

Import utility script notebook with data collection functions. See notebook for details: [Home Credit: baseline - DATA](https://www.kaggle.com/code/andreynesterov/home-credit-baseline-data)

In [5]:
import home_credit_baseline_data as data_nb

In [6]:
# 既にモデルの精度検証用にいい感じに整形してくれたベースライン訓練用データがhome_credit_baseline_dataに作られており、ここでは全ての特徴量を使うようなテーブル（train_full）を利用している
# 新しい特徴量生成などをひとまず考えないのであればこのテーブルを利用すれば色々なモデルのアンサンブルなどがとりあえずは試せそう？
train_df = pd.read_parquet('/kaggle/usr/lib/home_credit_baseline_data/train_full.parquet')
cat_cols = list(train_df.select_dtypes("category").columns)
display(train_df)

,case_id,WEEK_NUM,target,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtcount_4527229L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,isbidproduct_1095L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_60

In [7]:
# 上記のうちのカテゴリカル変数を確認
display(cat_cols)

['description_5085714M',
 'education_1103M',
 'education_88M',
 'maritalst_385M',
 'maritalst_893M',
 'requesttype_4525192L',
 'bankacctype_710L',
 'cardtype_51L',
 'credtype_322L',
 'disbursementtype_67L',
 'inittransactioncode_186L',
 'lastapprcommoditycat_1041M',
 'lastcancelreason_561M',
 'lastrejectcommoditycat_161M',
 'lastrejectcommodtypec_5251769M',
 'lastrejectreason_759M',
 'lastrejectreasonclient_4145040M',
 'lastst_736L',
 'paytype1st_925L',
 'paytype_783L',
 'twobodfilling_608L',
 'typesuite_864L',
 'max_cancelreason_3545846M',
 'max_education_1138M',
 'max_postype_4733339M',
 'max_rejectreason_755M',
 'max_rejectreasonclient_4145042M',
 'min_cancelreason_3545846M',
 'min_education_1138M',
 'min_postype_4733339M',
 'min_rejectreason_755M',
 'min_rejectreasonclient_4145042M',
 'first_cancelreason_3545846M',
 'first_education_1138M',
 'first_postype_4733339M',
 'first_rejectreason_755M',
 'first_rejectreasonclient_4145042M',
 'last_cancelreason_3545846M',
 'last_education_11

In [8]:
# テストデータを取得する部分
# home_credit_baseline_dataのprepare_df関数を見ると分かるが、train_colsに指定したカラムを渡すとそのtrain_colsに含まれるカラムを用いてテストデータをフィルタリングするため、訓練データに応じて必要なカラムだけテストデータに残してくれる
# つまり、先ほどの訓練データ取得＋このテストデータ取得処理のセットだけでいい感じに訓練データとテストデータをまとめて得られる（訓練データにtrain_full以外のものを指定してもこのセルでそれに合ったカラムだけテストデータに持ってきてくれる）
test_df = data_nb.prepare_df(data_nb.CFG.test_dir, cat_cols=cat_cols, mode="test", train_cols=train_df.columns)
display(test_df)

Feature engeneering...
  feats_df shape:	 (10, 926)
Filter cols...
  feats_df shape:	 (10, 515)
Convert to pandas...


,case_id,WEEK_NUM,month_decision,weekday_decision,assignmentdate_238D,assignmentdate_4527235D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtcount_4527229L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,secondquarter_766L,thirdquarter_1082L,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastinstal40dpd_247D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,disbursedcredamount_1113A,disbursementtype_67L,downpmt_116A,dtlastpmtallstes_4499206D,eir_270L,equalitydataagreement_891L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,inittransactionamount_650A,inittransactioncode_186L,interestrate_311L,isbidproduct_1095L,isdebitcard_729L,lastactivateddate_801D,lastapplicationdate_877D,lastapprcommoditycat_1041M,lastapprcredamount_781A,lastapprdate_640D,lastcancelreason_561M,lastdelinqdate_224D,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectcredamount_222A,lastrejectdate_50D,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,maininc_215A,mastercontrelectronic_519L,mastercontrexist_109L,maxannuity_159A,maxdbddpdlast1m_3658939P,maxdbddpdtollast12m_3658940P,maxdbddpdtollast6m_4187119P,maxdebt4_972A,maxdpdfrom6mto36m_3546853P,maxdpdinstldate_3546855D,maxdpdinstlnum_3546846P,maxdpdlast12m_727P,maxdpdlast24m_143P,maxdpdlast3m_392P,maxdpdlast6m_474P,maxdpdlast9m_1059P,maxdpdtolerance_374P,maxinstallast24m_3658928A,maxlnamtstart6m_4525199A,maxoutstandbalancel12m_4187113A,maxpmtlast3m_4525190A,mindbddpdlast24m_3658935P,mindbdtollast24m_4525191P,mobilephncnt_593L,monthsannuity_845L,numactivecreds_622L,numactivecredschannel_414L,numactiverelcontr_750L,numcontrs3months_479L,numincomingpmts_3546848L,numinstlallpaidearly3d_817L,numinstls_657L,numinstlsallpaid_934L,numinstlswithdpd10_728L,numinstlswithdpd5_4187116L,numinstlswithoutdpd_562L,numinstmatpaidtearly2d_4499204L,numinstpaid_4499208L,numinstpaidearly3d_3546850L,numinstpaidearly3dest_4493216L,numinstpaidearly5d_1087L,numinstpaidearly5dest_4493211L,numinstpaidearly5dobd_4499205L,numinstpaidearly_338L,numinstpaidearlyest_4493214L,numinstpaidlastcontr_4325080L,numinstpaidlate1d_3546852L,numinstregularpaid_973L,numinstregularpaidest_4493210L,numinsttopaygr_769L,numinsttopaygrest_4493213L,numinstunpaidmax_3546851L,numinstunpaidmaxest_4493212L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pctinstlsallpaidearl3d_427L,pctinstlsallpaidlat10d_839L,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,type

## Drop columns
- 不要なカラムをドロップし、最終的にできた訓練・テストテーブルを最初のメモリ圧縮用関数にかけてモデル入力用に小さくしている

In [9]:
# mean_, first_,それぞれのプレフィックスのカラムをドロップしている。
# 個人の平均の〜や最初の〜といった情報は使わないらしい
drop_cols = []
drop_cols_startwith = ["mean_", "first_"]
for name_prefix in drop_cols_startwith:
    cols_names = train_df.columns[train_df.columns.str.startswith(name_prefix)]
    drop_cols += cols_names.to_list()
display(drop_cols)
display(len(drop_cols))

['mean_actualdpd_943P',
 'mean_annuity_853A',
 'mean_credacc_actualbalance_314A',
 'mean_credacc_credlmt_575A',
 'mean_credacc_maxhisbal_375A',
 'mean_credacc_minhisbal_90A',
 'mean_credamount_590A',
 'mean_currdebt_94A',
 'mean_downpmt_134A',
 'mean_mainoccupationinc_437A',
 'mean_maxdpdtolerance_577P',
 'mean_outstandingdebt_522A',
 'mean_revolvingaccount_394A',
 'mean_approvaldate_319D',
 'mean_creationdate_885D',
 'mean_dateactivated_425D',
 'mean_dtlastpmt_581D',
 'mean_dtlastpmtallstes_3545839D',
 'mean_employedfrom_700D',
 'mean_firstnonzeroinstldate_307D',
 'mean_amount_4527230A',
 'mean_recorddate_4527225D',
 'mean_amount_4917619A',
 'mean_deductiondate_4917603D',
 'mean_pmtamount_36A',
 'mean_processingdate_168D',
 'mean_mainoccupationinc_384A',
 'mean_birth_259D',
 'mean_empl_employedfrom_271D',
 'mean_amount_416A',
 'mean_openingdate_313D',
 'mean_openingdate_857D',
 'first_actualdpd_943P',
 'first_annuity_853A',
 'first_credacc_credlmt_575A',
 'first_credamount_590A',
 'fi

98

In [10]:
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)

In [11]:
train_df.shape

(1526659, 418)

In [12]:
train_df = data_nb.reduce_mem_usage(train_df, float16_as32=False)

Memory usage of dataframe is 2105.33 MB
Memory usage after optimization is: 1272.53 MB
Decreased by 39.6%


In [13]:
test_df = data_nb.reduce_mem_usage(test_df, float16_as32=False)

Memory usage of dataframe is 0.04 MB
Memory usage after optimization is: 0.03 MB
Decreased by 21.3%


In [14]:
gc.collect()

8

# Training
- tabular auto MLで先ほど整形したデータで訓練を行う

In [15]:
# TabularAutoMLに指定するパラメータ
# sklearnのメソッドに入れるパラメータみたいなもの
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 2024
TIMEOUT = 10000
ADVANCED_ROLES = False
USE_QNT = False
USE_PLR = False
TRAIN_BS = 128
EPOCHS = 5
TARGET_NAME = 'target'

np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [16]:
# おそらく予測タスクの種類を指定できる。今回は二値分類なのでこの関数はおそらく呼ばれていない
def map_class(x, task, reader):
    if task.name == 'multiclass':
        return reader[x]
    else:
        return x

mapped = np.vectorize(map_class)

# automlのtaskインスタンスで目的に応じた評価関数を用意してくれている部分。今回は二値分類なのでaucを返す
def score(task, y_true, y_pred):
    if task.name == 'binary':
        return roc_auc_score(y_true, y_pred)
    elif task.name == 'multiclass':
        return log_loss(y_true, y_pred)
    elif task.name == 'reg' or task.name == 'multi:reg':
        return mean_absolute_error(y_true, y_pred)
    else:
        raise 'Task is not correct.'

# モデルの予測値を返す関数（今回は二値分類なのでbinaryが呼ばれ、aucを計算するときのpredict_probaみたいなやつを返す）
def take_pred_from_task(pred, task):
    if task.name == 'binary' or task.name == 'reg':
        return pred[:, 0]
    elif task.name == 'multiclass' or task.name == 'multi:reg':
        return pred
    else:
        raise 'Task is not correct.'

# autoMLの中身でnnも使っているため、そのときの連続変数の扱い方。今回はUSE_PLR=Falseなのでそのまま使うが、Trueになるとplrを返して擬似ラベルを付与する
def use_plr(USE_PLR):
    if USE_PLR:
        return "plr"
    else:
        return "cont"

In [17]:
# 今回のauto MLの予測タスクの種類と評価指標を指定する部分。（二値分類、ログ損失、auc）
task = Task(
    'binary', 
    loss = 'logloss', 
    metric = 'auc'
)

# 目的変数のカラム名、グループ化の基準とするカラム名、ドロップするカラム名をそれぞれ指定
# おそらくバリデーション時にWEEK_NUMの順で訓練→テストデータにしないとリークして過学習する（例えばバリデーション時に90週目のデータを訓練に使って45週目のデータを予測するのは不適切）
roles = {
    'target': TARGET_NAME,
    'group': "WEEK_NUM",
    'drop': ['case_id', "WEEK_NUM"],
}

In [18]:
%%time
# 中で用いるnnの種類を選択している
# 要はかなりブラックボックスに利用できるnnライブラリで、TabularAutoMLメソッドで簡単にdenselightというアルゴリズムのnnを呼び出して訓練している
# 実は中でこのnnと他の木系のやつとかもアンサンブルしているかもしれないがそこまでは読み取れなかった
# denselightは割と軽量で計算リソースに限りがある場合に使うことが多そう
# 参考：https://lightautoml.readthedocs.io/en/latest/pages/tutorials/Tutorial_9_neural_networks.html#:~:text=1.2%20Dense%20Light%20(%22denselight%22)
algo = 'denselight'

automl_model = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    gpu_ids='0',
    general_params = {"use_algos": [[algo]]}, # ['nn', 'mlp', 'dense', 'denselight', 'resnet', 'snn', 'node', 'autoint', 'fttransformer'] or custom torch model
    nn_params = {
        "n_epochs": EPOCHS, 
        "bs": TRAIN_BS, 
        "num_workers": 0, 
        "path_to_save": None, 
        "freeze_defaults": True,
        "cont_embedder": use_plr(USE_PLR),
    },
    nn_pipeline_params = {
        "use_qnt": USE_QNT, 
        "use_te": False
    },
    reader_params = {
        'n_jobs': N_THREADS, 
        'cv': N_FOLDS, 
        'random_state': RANDOM_STATE, 
        'advanced_roles': ADVANCED_ROLES
    },
)

oof_pred = automl_model.fit_predict(train_df, roles = roles, verbose = 3)

[19:52:05] Stdout logging level is INFO3.
[19:52:05] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[19:52:05] Task: binary

[19:52:05] Start automl preset with listed constraints:
[19:52:05] - time: 10000.00 seconds
[19:52:05] - CPU: 4 cores
[19:52:05] - memory: 16 GB

[19:52:05] Train data shape: (1526659, 418)

[19:52:13] Layer 1 train process start. Time left 9992.34 secs
[19:53:49] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 ...
[19:53:53] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_TorchNN_denselight_0 =====
[19:57:29] Epoch: 0, train loss: 0.11989961564540863, val loss: 0.11708773672580719, val metric: 0.8196523848365491
[20:00:37] Epoch: 1, train loss: 0.11724480986595154, val loss: 0.11624324321746826, val metric: 0.8245943070368806
[20:03:45] Epoch: 2, train loss: 0.11619338393211365, val loss: 0.11582745611667633, val metric: 0.826457808453581
[20:06:53] Epoch: 3, train loss: 0.11530479043722153, val lo

In [19]:
oof_pred.data

array([[0.04300056],
       [0.02889601],
       [0.04340824],
       ...,
       [0.05523512],
       [0.0014535 ],
       [0.00635855]], dtype=float32)

In [20]:
# 訓練データに対するaucだと0.83くらい
oof = score(
    task,
    mapped(train_df[TARGET_NAME].values, task, automl_model.reader.class_mapping),
    take_pred_from_task(oof_pred.data, task)
)
print("CV roc_auc_oof: ", oof)

CV roc_auc_oof:  0.8290582712163738


In [21]:
# 今回出した予測値を保存
joblib.dump(oof_pred.data, 'denselight_oof_preds.pkl')

['denselight_oof_preds.pkl']

In [22]:
# 今回訓練したモデルを保存（このモデルをhome-credit-baseline-inferenceでインポートしてmodel3として利用する）
joblib.dump(automl_model, 'denselight_model.pkl')

['denselight_model.pkl']

In [23]:
# 今回の訓練データをカテゴリカルカラム・ドロップしたカラム情報とセットで保存
joblib.dump((train_df.columns, cat_cols, drop_cols), "train_cat_columns.pkl")

['train_cat_columns.pkl']

In [24]:
del oof_pred
gc.collect()

0

# Prediction
- 先ほど訓練したモデルでテストデータの予測を行って提出しているだけの部分

In [25]:
def predict_proba_in_batches(model, data, batch_size=100000):
    num_samples = len(data)
    num_batches = int(np.ceil(num_samples / batch_size))
    probabilities = np.zeros((num_samples,))

    for batch_idx in range(num_batches):
        print(f"Processing batch: {batch_idx+1}/{num_batches}")
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, num_samples)
        X_batch = data.iloc[start_idx:end_idx]
        batch_probs = model.predict(X_batch).data.squeeze()
        probabilities[start_idx:end_idx] = batch_probs
        gc.collect()

    return probabilities

In [26]:
X_test = test_df.drop(columns=["WEEK_NUM"])
X_test = X_test.set_index("case_id")
print("X_test shape: ", X_test.shape)

y_pred = pd.Series(predict_proba_in_batches(automl_model, X_test), index=X_test.index)
y_pred[:10]

X_test shape:  (10, 415)
Processing batch: 1/1


case_id
57543    0.180577
57549    0.095292
57551    0.090172
57552    0.012572
57569    0.007465
57630    0.012604
57631    0.018216
57632    0.016031
57633    0.015755
57634    0.021687
dtype: float64

# Submission

In [27]:
subm_df = pd.read_csv(data_nb.CFG.root_dir / "sample_submission.csv")
subm_df = subm_df.set_index("case_id")
subm_df["score"] = y_pred

In [28]:
print("Check null: ", subm_df["score"].isnull().any())
subm_df.head()

Check null:  False


,score
case_id,
57543,0.180577
57549,0.095292
57551,0.090172
57552,0.012572
57569,0.007465


In [29]:
subm_df.to_csv("submission.csv")